### When running this notebook via the Galaxy portal
You can access your data via the dataset number. Using a Python kernel, you can access dataset number 42 with ``handle = open(get(42), 'r')``.
To save data, write your data to a file, and then call ``put('filename.txt')``. The dataset will then be available in your galaxy history.
<br><br>Note that if you are putting/getting to/from a different history than your default history, you must also provide the history-id.
<br><br>More information including available galaxy-related environment variables can be found at https://github.com/bgruening/docker-jupyter-notebook. This notebook is running in a docker container based on the Docker Jupyter container described in that link.


# Fourlepton analysis  (Python) 


This is an example of a simple dilepton analysis, quite similar to the notebook called "Dilepton_analysis_noData.ipynb", but with some differences. The most obvious difference is that we here also include real data. This example also has a slightly more advanced event selection.  

**Notice:** This is *only an example* on how to do this. Feel free to be creative, and to find better and/or more elegant ways of doing the various steps! 

In [1]:
import ROOT as R
import import_ipynb
import setPath
from os import listdir
from os.path import isfile, join
from Input.OpenDataPandaFramework13TeV import *
%jsroot on

Welcome to JupyROOT 6.24/02
importing Jupyter notebook from setPath.ipynb
importing Jupyter notebook from /storage/galaxy/jobs_directory/002/2751/working/jupyter/Input/OpenDataPandaFramework13TeV.ipynb
This library contains handy functions to ease the access and use of the 13TeV ATLAS OpenData release

getBkgCategories()
	 Dumps the name of the various background cataegories available 
	 as well as the number of samples contained in each category.
	 Returns a vector with the name of the categories

getSamplesInCategory(cat)
	 Dumps the name of the samples contained in a given category (cat)
	 Returns dictionary with keys being DSIDs and values physics process name from filename.

getMCCategory()
	 Returns dictionary with keys DSID and values MC category

initialize(indir)
	 Collects all the root files available in a certain directory (indir)



Setting luminosity to 10064 pb^-1

###############################
#### Background categories ####
###############################
Category    

## 1. Reading the dataset

Set the analaysis to run (*1largeRjet1lep*, *1lep1tau*, *3lep*, *exactly2lep*, *GamGam*, *2lep*, *4lep*)

Set the directory where you have downloaded the ATLAS OpenData samples you want to run over

In [2]:
opendatadir = "/storage/shared/data/fys5555/ATLAS_opendata/"
analysis = "4lep"

In [3]:
background = R.TChain("mini")
data = R.TChain("mini")

A list of all the background samples, category and their IDs can be found in **Infofile.txt**. The cross-section, efficiencies etc. needed for scaling are stored in the **Files_<---->**. We read these files and add all the samples to the TChain. We also (for later convenience) make a vector containing the dataset IDs. 

In [4]:
mcfiles = initialize(opendatadir+"/"+analysis+"/MC")
datafiles = initialize(opendatadir+"/"+analysis+"/Data")
allfiles = z = {**mcfiles, **datafiles}
Backgrounds = getBkgCategories() 
print(Backgrounds)
skip = [2,3]
for i in skip:
    Backgrounds.pop(i)
    
print(Backgrounds)

###############################
#### Background categories ####
###############################
Category             N(samples)
-------------------------------
Diboson                       9
Higgs                        11
Wjets                        42
Wjetsincl                     6
Zjets                        42
Zjetsincl                     3
singleTop                     6
['Diboson', 'Higgs', 'Wjets', 'Wjetsincl', 'Zjets', 'Zjetsincl', 'singleTop']
['Diboson', 'Higgs', 'Wjetsincl', 'Zjetsincl', 'singleTop']


In [5]:
MCcat = {}
for cat in allfiles:
    for dsid in allfiles[cat]["dsid"]:
        try:
            MCcat[int(dsid)] = cat
        except:
            continue

In [6]:
dataset_IDs = []
background.Reset()
for b in Backgrounds:
    i = 0
    if not b in mcfiles.keys(): continue
    for mc in mcfiles[b]["files"]:
        if not os.path.isfile(mc): continue
        try:
            dataset_IDs.append(int(mcfiles[b]["dsid"][i]))
            background.Add(mc)
        except:
            print("Could not get DSID for %s. Skipping"%mc)
        i += 1
nen = background.GetEntries()
print("Added %i entries for backgrounds"%(nen))

Added 951517 entries for backgrounds


In [7]:
data.Reset(); 
for d in datafiles["data"]["files"]:  
    if not os.path.isfile(d): continue
    data.Add(d)
nen = data.GetEntries()
print("Added %i entries for backgrounds"%(nen))

Added 832 entries for backgrounds


## 2. Making (a lot of) histograms

Now that we have read our dataset we want to start analyzing the data. To do so we need to put the data into histograms. For reasons that will become clear later in the analysis we must (for each variable) make one histogram per dataset ID. (We have 31 background samples, so if we want to study 10 variables we have to make 310 histograms!) For best dealing with all these histograms we can use dictionaries (Python) or maps (C++). 

In [8]:
hist_mll = {}; hist_lep_pt = {};  hist_dilep1 = {}; hist_dilep2 = {};
binsize = 24
min_v = 80
max_v = 170

bin_pt = 5
min_pt = 0
max_pt = 200



In [9]:
for i in dataset_IDs: 
    hist_mll[i] = R.TH1F() 
    hist_lep_pt[i] = R.TH1F()

    hist_dilep1[i] = R.TH1F()
    hist_dilep2[i] = R.TH1F()

In [10]:
for i in dataset_IDs: 
    hist_mll[i].SetNameTitle("hist_mllll", "Invariant mass"); 
    hist_lep_pt[i].SetNameTitle("hist_lep_pt", "Lepton pT"); 
    
    hist_dilep1[i].SetNameTitle("hist_dilep1", "Invariant mass");
    hist_dilep2[i].SetNameTitle("hist_dilep2", "Invariant mass");
    
    hist_mll[i].SetBins(binsize,min_v,max_v); 
    hist_dilep1[i].SetBins(binsize+5,min_v-60,max_v-30); 
    hist_dilep2[i].SetBins(binsize+5,min_v-60,max_v-30); 
    hist_lep_pt[i].SetBins(bin_pt,min_pt,max_pt);
    

For data it is only necessary with one histogram for each variable: 

In [11]:
hist_mll_d = R.TH1F(); 
hist_lep_pt_d = R.TH1F(); 

hist_dilep1_d = R.TH1F(); 
hist_dilep2_d = R.TH1F();

In [12]:
hist_mll_d.SetNameTitle("hist_mllll", "Invariant mass"); 
hist_lep_pt_d.SetNameTitle("hist_lep_pt", "Lepton pT"); 

hist_dilep1_d.SetNameTitle("hist_dilep1", "Invariant mass");
hist_dilep2_d.SetNameTitle("hist_dilep2", "Invariant mass");
hist_mll_d.SetBins(binsize,min_v,max_v); 
hist_dilep1_d.SetBins(binsize+5,min_v-60,max_v-30); 
hist_dilep2_d.SetBins(binsize+5,min_v-60,max_v-30); 
hist_lep_pt_d.SetBins(bin_pt,min_pt,max_pt);


In [13]:
# Retrieve lumi from library
%store -r lumi

no stored variable or alias lumi


### 2.1 Fill the histograms 
We can now loop over all events in our dataset, implement desired cuts, and fill the histograms we created above. In this example we choose only events containing exactly to same flavour leptons with opposite charge (i.e. $e^+e^-$ or $\mu^+\mu^-$). 
Before starting the loop we extract the total number of entries (events) in the TChain. We also make [TLorentzVector](https://root.cern.ch/doc/master/classTLorentzVector.html)s, which are very practical for handling the kinematics of the leptons, e.g. calculating the invariant mass of the two leptons. 

In [99]:
for i in dataset_IDs: 
    hist_mll[i].Reset(); 
    hist_lep_pt[i].Reset(); 
   
    hist_dilep1[i].Reset();
    hist_dilep2[i].Reset();

In [100]:
hist_mll_d.Reset(); 
hist_lep_pt_d.Reset(); 

hist_dilep1_d.Reset();
hist_dilep2_d.Reset();

In [101]:
l1 = R.TLorentzVector() 
l2 = R.TLorentzVector() 
l3 = R.TLorentzVector() 
l4 = R.TLorentzVector()

fourleptons = R.TLorentzVector() 
dilepton1 = R.TLorentzVector()
dilepton2 = R.TLorentzVector()

p1 = R.TLorentzVector()
p2 = R.TLorentzVector()
p3 = R.TLorentzVector()
p4 = R.TLorentzVector()


def get_mass(lep_pair):
    a, b = lep_pair
    dilep = a + b
    mass = dilep.M()
    return mass


This is the cell where the analysis is performed. Note that the cell needs to be run twice:

1. with data = 0 to run over MC
2. with data = 1 to run over data

Note that the MC running takes ~5 minutes for 3lep analysis. Much(!!!) more time for e.g. 2lep analysis! Data running is relatively fast for 3lep. 

In [103]:
%%time
import time
import numpy as np 
isData = 0; 

if isData == 1: ds = data 
else: ds = background    

legal_flavor_tot = [13*4, 11*4, 11*2 + 13*2]
z_mass = 91.1876
i = 0   
for event in ds: 
    
    if i%100000 == 0 and i>0: 
        print("Total events %i/%i"%(i,ds.GetEntries()))
    i += 1 
    ## Data quality cuts: 
    
    
    #if i > 10000: break
    
    #if ds.passGRL == 0: continue
    #if ds.hasGoodVertex == 0: continue
    #if(trigM == 0 && trigE == 0){ continue; } 

    ## Event selection:
    
    ## Cut #1: Require (exactly) 2 leptons
    if not ds.lep_n == 4: continue
    ## Cut #2: Require opposite charge
    if not ds.lep_charge[0] + ds.lep_charge[1] + ds.lep_charge[2] + ds.lep_charge[3] == 0 : continue
    
    ## Cut #3: Require same flavour (2 electrons or 2 muons)
    if not ds.lep_type[0] + ds.lep_type[1] + ds.lep_type[2] + ds.lep_type[3] in legal_flavor_tot: continue

    
    ## Require "good leptons": 
    
    ## Set Lorentz vectors: 
    l1.SetPtEtaPhiE(ds.lep_pt[0]/1000., ds.lep_eta[0], ds.lep_phi[0], ds.lep_E[0]/1000.);
    l2.SetPtEtaPhiE(ds.lep_pt[1]/1000., ds.lep_eta[1], ds.lep_phi[1], ds.lep_E[1]/1000.);
    l3.SetPtEtaPhiE(ds.lep_pt[2]/1000., ds.lep_eta[2], ds.lep_phi[2], ds.lep_E[2]/1000.);
    l4.SetPtEtaPhiE(ds.lep_pt[3]/1000., ds.lep_eta[3], ds.lep_phi[3], ds.lep_E[3]/1000.);
    ## Variables are stored in the TTree with unit MeV, so we need to divide by 1000 
    ## to get GeV, which is a more practical and commonly used unit. 
    
    if abs(ds.lep_z0[0]*np.sin(l1.Theta())) > 0.5: continue
    #if abs(ds.lep_z0[1]*np.sin(l2.Theta())) > 0.5: continue
    #if abs(ds.lep_z0[2]*np.sin(l3.Theta())) > 0.5: continue
    #if abs(ds.lep_z0[3]*np.sin(l4.Theta())) > 0.5: continue
    
    
    if ds.lep_etcone20[0]/ds.lep_pt[0] > 0.15: continue
    if ds.lep_ptcone30[0]/ds.lep_pt[0] > 0.15: continue
    #if not (ds.lep_flag[0] & 512): continue
        
    
    if ds.lep_etcone20[1]/ds.lep_pt[1] > 0.3: continue
    if ds.lep_ptcone30[1]/ds.lep_pt[1] > 0.3: continue
    #if not (ds.lep_flag[1] & 512): continue
    
    if ds.lep_etcone20[2]/ds.lep_pt[2] > 0.3: continue
    if ds.lep_ptcone30[2]/ds.lep_pt[2] > 0.3: continue
    #if not (ds.lep_flag[0] & 512): continue
        
    
    if ds.lep_etcone20[3]/ds.lep_pt[3] > 0.3: continue
    if ds.lep_ptcone30[3]/ds.lep_pt[3] > 0.3: continue
    #if not (ds.lep_flag[1] & 512): continue

    
    if ds.lep_type[0] == 11:
        if abs(ds.lep_eta[0]) > 2.47: continue
        if ds.lep_trackd0pvunbiased[0]/ds.lep_tracksigd0pvunbiased[0] > 5: continue
    elif ds.lep_type[0] == 13:
        if abs(ds.lep_eta[0]) > 2.5: continue
        if ds.lep_trackd0pvunbiased[0]/ds.lep_tracksigd0pvunbiased[0] > 3: continue
    if abs(ds.lep_eta[0]) < 1.52 and abs(ds.lep_eta[0]) > 1.37: continue
    if ds.lep_pt[0]/1000.0 < 25: continue
    
    
    if ds.lep_type[1] == 11:
        if abs(ds.lep_eta[1]) > 2.47: continue
        if ds.lep_trackd0pvunbiased[1]/ds.lep_tracksigd0pvunbiased[1] > 5: continue
    elif ds.lep_type[1] == 13:
        if abs(ds.lep_eta[1]) > 2.5: continue
        if ds.lep_trackd0pvunbiased[1]/ds.lep_tracksigd0pvunbiased[1] > 3: continue
    if abs(ds.lep_eta[1]) < 1.52 and abs(ds.lep_eta[1]) > 1.37: continue
    if ds.lep_pt[1]/1000.0 < 15: continue
    
    
    if ds.lep_type[2] == 11:
        if abs(ds.lep_eta[2]) > 2.47: continue
        if ds.lep_trackd0pvunbiased[2]/ds.lep_tracksigd0pvunbiased[2] > 5: continue
    elif ds.lep_type[2] == 13:
        if abs(ds.lep_eta[2]) > 2.5: continue
        if ds.lep_trackd0pvunbiased[2]/ds.lep_tracksigd0pvunbiased[2] > 3: continue
    if abs(ds.lep_eta[2]) < 1.52 and abs(ds.lep_eta[2]) > 1.37: continue
    if ds.lep_pt[2]/1000.0 < 10: continue
    
    
    if ds.lep_type[3] == 11:
        if abs(ds.lep_eta[3]) > 2.47: continue
        if ds.lep_trackd0pvunbiased[3]/ds.lep_tracksigd0pvunbiased[3] > 5: continue
    elif ds.lep_type[3] == 13:
        if abs(ds.lep_eta[3]) > 2.5: continue
        if ds.lep_trackd0pvunbiased[3]/ds.lep_tracksigd0pvunbiased[3] > 3: continue
    if abs(ds.lep_eta[3]) < 1.52 and abs(ds.lep_eta[3]) > 1.37: continue
    if ds.lep_pt[3]/1000.0 < 7: continue
        
    
    
    ## Set Lorentz vectors: 
    l1.SetPtEtaPhiE(ds.lep_pt[0]/1000., ds.lep_eta[0], ds.lep_phi[0], ds.lep_E[0]/1000.);
    l2.SetPtEtaPhiE(ds.lep_pt[1]/1000., ds.lep_eta[1], ds.lep_phi[1], ds.lep_E[1]/1000.);
    l3.SetPtEtaPhiE(ds.lep_pt[2]/1000., ds.lep_eta[2], ds.lep_phi[2], ds.lep_E[2]/1000.);
    l4.SetPtEtaPhiE(ds.lep_pt[3]/1000., ds.lep_eta[3], ds.lep_phi[3], ds.lep_E[3]/1000.);
    ## Variables are stored in the TTree with unit MeV, so we need to divide by 1000 
    ## to get GeV, which is a more practical and commonly used unit. 
    
    
    
    if ds.lep_type[0] + ds.lep_type[1] + ds.lep_type[2] + ds.lep_type[3] == 4*11 or  ds.lep_type[0] + ds.lep_type[1] + ds.lep_type[2] + ds.lep_type[3] == 4 * 13:
    ## Find dilepton pair mass
        if ds.lep_charge[0] == ds.lep_charge[1]:
            lep_list = [l1, l2, l3, l4]
            mass = []
            diff = 1000
            leg1 = l1
            leg2 = l4
            
            p1 = l1 + l3
            p2 = l1 + l4
            p3 = l2 + l3
            p4 = l2 + l4
            
            m1 = p1.M() - z_mass; 
            mass.append((m1, l1, l3));
            m2 = p2.M() - z_mass; 
            mass.append((m2, l1, l4));
            m3 = p3.M() - z_mass; 
            mass.append((m3, l2, l3));
            m4 = p4.M() - z_mass; 
            mass.append((m4, l2, l4));
            
            for combo in mass:
                mass_diff, legg1, legg2 = combo
                if abs(mass_diff) < diff:
                    diff == abs(mass_diff)
                    leg1 = legg1
                    leg2 = legg2
                    
            lep_list.remove(leg1)
            lep_list.remove(leg2)
            
            dilepton1 = leg1 + leg2
            dilepton2 = lep_list[0] + lep_list[1]
            
        elif ds.lep_charge[0] == ds.lep_charge[2]:
            lep_list = [l1, l2, l3, l4]
            mass = []
            diff = 1000
            leg1 = l1
            leg2 = l2
            
            p1 = l1 + l2
            p2 = l1 + l4
            p3 = l2 + l3
            p4 = l3 + l4
            
            m1 = p1.M() - z_mass; 
            mass.append((m1, l1, l3));
            m2 = p2.M() - z_mass; 
            mass.append((m2, l1, l4));
            m3 = p3.M() - z_mass; 
            mass.append((m3, l2, l3));
            m4 = p4.M() - z_mass; 
            mass.append((m4, l2, l4));
            
            for combo in mass:
                mass_diff, legg1, legg2 = combo
                if abs(mass_diff) < diff:
                    diff == abs(mass_diff)
                    leg1 = legg1
                    leg2 = legg2
                    
            lep_list.remove(leg1)
            lep_list.remove(leg2)
            
            dilepton1 = leg1 + leg2
            dilepton2 = lep_list[0] + lep_list[1]
            
        elif ds.lep_charge[0] == ds.lep_charge[3]:
            lep_list = [l1, l2, l3, l4]
            mass = []
            diff = 1000
            leg1 = l1
            leg2 = l3
            
            p1 = l1 + l2
            p2 = l1 + l3
            p3 = l2 + l4
            p4 = l4 + l3
            
            m1 = p1.M() - z_mass; 
            mass.append((m1, l1, l3));
            m2 = p2.M() - z_mass; 
            mass.append((m2, l1, l4));
            m3 = p3.M() - z_mass; 
            mass.append((m3, l2, l3));
            m4 = p4.M() - z_mass; 
            mass.append((m4, l2, l4));
            
            for combo in mass:
                mass_diff, legg1, legg2 = combo
                
                if abs(mass_diff) < diff:
                    diff == abs(mass_diff)
                    leg1 = legg1
                    leg2 = legg2
                    
            lep_list.remove(leg1)
            lep_list.remove(leg2)
            
            
            
            dilepton1 = leg1 + leg2
            dilepton2 = lep_list[0] + lep_list[1]
    
    else: # Check who belongs to who
        if ds.lep_type[0] == ds.lep_type[1]:
            dilepton1 = l1 + l2
            dilepton2 = l3 + l4
        elif ds.lep_type[0] == ds.lep_type[2]:
            dilepton1 = l1 + l3
            dilepton2 = l2 + l4
        elif ds.lep_type[0] == ds.lep_type[3]:
            dilepton1 = l1 + l4
            dilepton2 = l2 + l3
            
    fourleptons = l1 + l2 + l3 + l4;   

    if isData == 1:
        hist_mll_d.Fill(fourleptons.M());
        hist_lep_pt_d.Fill(l1.Pt());
        hist_lep_pt_d.Fill(l2.Pt());
        hist_lep_pt_d.Fill(l3.Pt());
        hist_lep_pt_d.Fill(l4.Pt());
        
        hist_dilep1_d.Fill(dilepton1.M());
        hist_dilep2_d.Fill(dilepton2.M());
    else: 
        W = ((ds.mcWeight)*(ds.scaleFactor_PILEUP)*
             (ds.scaleFactor_ELE)*(ds.scaleFactor_MUON)*
             (ds.scaleFactor_LepTRIGGER))*((ds.XSection*lumi)/ds.SumWeights)
        hist_mll[ds.channelNumber].Fill(fourleptons.M(), W);
        hist_lep_pt[ds.channelNumber].Fill(l1.Pt(), W);
        hist_lep_pt[ds.channelNumber].Fill(l2.Pt(), W); 
        hist_lep_pt[ds.channelNumber].Fill(l3.Pt(), W);
        hist_lep_pt[ds.channelNumber].Fill(l4.Pt(), W);
        
        hist_dilep1[ds.channelNumber].Fill(dilepton1.M(), W);
        hist_dilep2[ds.channelNumber].Fill(dilepton2.M(), W);
        
print("Done!")
if isData == 0:
    print("Remebered to run over data? No? Set data = 1 at the top and run again")
else:
    print("Remebered to run over MC? No? Set data = 0 at the top and run again")

Total events 100000/951517
Total events 200000/951517
Total events 300000/951517
Total events 400000/951517
Total events 500000/951517
Total events 600000/951517
Total events 700000/951517
Total events 800000/951517
Total events 900000/951517
Done!
Remebered to run over data? No? Set data = 1 at the top and run again
CPU times: user 4min 28s, sys: 536 ms, total: 4min 28s
Wall time: 4min 29s


## 3. Scale and classify the histograms (MC only) 

Before we are ready to make plots we need to do some further processing of the histograms we made above. The information necessary for doing the two steps below is found in the file **Infofile.txt**.   
1. We need to **scale** the histograms to the right cross section and luminosity. Why? When making the MC samples a certain number of events is simulated, which will usually not correspond to the number of events in our data. The expected number of events from a certain kind of process is given by $N=\sigma L$, where $\sigma$ is the cross section and $L$ is the integrated luminosity. Therefore we need to scale each histogram by a scale factor <br> <br>
$$sf = \frac{N}{N_{MC}} = \frac{ \sigma L }{N_{MC}},$$ <br>  where $N_{MC}$ is the number of generated MC events.  <br> <br>
2. We also need to **classify** the background processes into different categories. This is necessary when we eventually want to make the characteristic colorful background plots you might have seen before.  

### 3.1 Make new histograms 
Maybe a bit depressingly we have to make a set of new histograms, this time corresponding to the different background categories, instead of the dataset IDs. Notice that these new histograms are made in a very similar way as above, i.e. with the same range and binning. 

In [76]:
H_mll = {}; H_lep_pt = {}; H_dilep1 = {}; H_dilep2 = {};

In [77]:
L = 10.06

In [78]:
for i in Backgrounds: 
    H_mll[i] = R.TH1F() 
    H_lep_pt[i] = R.TH1F() 
    
    H_dilep1[i] = R.TH1F()
    H_dilep2[i] = R.TH1F()

In [79]:
for i in Backgrounds: 
    H_mll[i].SetNameTitle("hist_mllll", "Invariant mass"); 
    H_lep_pt[i].SetNameTitle("hist_lep_pt", "Lepton pT"); 
   
    H_dilep1[i].SetNameTitle("hist_dilep1", "Invariant mass"); 
    H_dilep2[i].SetNameTitle("hist_dilep2", "Invariant mass"); 
    
    H_mll[i].SetBins(binsize,min_v,max_v); 
    H_lep_pt[i].SetBins(bin_pt,min_pt,max_pt);
    
    H_dilep1[i].SetBins(binsize+5,min_v-60,max_v-30); 
    H_dilep2[i].SetBins(binsize+5,min_v-60,max_v-30); 

### 3.2 Scale and add histograms 
Now we read our info file, scale all (old) histograms, and then add them to the new histograms we just defined.  

In [80]:
for i in Backgrounds: 
    H_mll[i].Reset(); 
    H_lep_pt[i].Reset(); 
   
    H_dilep1[i].Reset();
    H_dilep2[i].Reset();

In [81]:
for dsid in hist_mll.keys():
    
    Type = MCcat[dsid]
    
    H_mll[Type].Add(hist_mll[dsid]); 
    H_lep_pt[Type].Add(hist_lep_pt[dsid]); 
    
    H_dilep1[Type].Add(hist_dilep1[dsid]);
    H_dilep2[Type].Add(hist_dilep2[dsid]);
    
infofile.close()

### 3.3 Color the histograms 
Make yet another map, this time containing the colors you want the backgrounds to have, and then set the colors of your histograms. Note that colors are defined by integers in ROOT. If you are not happy with the colors chosen below you can have look at the [TColor](https://root.cern.ch/doc/master/classTColor.html) class reference for more options. 

In [82]:
colors = {}

In [83]:
colors["Diboson"] = R.kGreen; 
colors["Zjets"] = R.kYellow; 
colors["ttbar"] = R.kRed;
colors["singleTop"] = R.kBlue-7; 
colors["Wjets"] = R.kBlue+3; 
colors["topX"] = R.kOrange+1; 
colors["Higgs"] = R.kMagenta; 
colors["Wjetsincl"] = R.kBlue-10;
colors["Zjetsincl"] = R.kYellow-9;

In [84]:
for h in Backgrounds: 
    H_mll[h].SetFillColor(colors[h]); 
    
    H_lep_pt[h].SetFillColor(colors[h]);
    H_dilep1[h].SetFillColor(colors[h]);
    H_dilep2[h].SetFillColor(colors[h]);
    
    H_mll[h].SetLineColor(colors[h]); 
   
    H_lep_pt[h].SetLineColor(colors[h]);
    H_dilep1[h].SetLineColor(colors[h]);
    H_dilep2[h].SetLineColor(colors[h]);

## 4. Stack and plot the histograms

Finally we have arrived to the part where we can plot the results of all the work done above. For each variable we need to stack the backgrounds on top of each other, which is done by using the [THStack](https://root.cern.ch/doc/master/classTHStack.html) class. In the example below we do this for two variables; invariant mass and missing $E_T$.   

In [85]:
stack_mll = R.THStack("Invariant mass 4l", "");
stack_met = R.THStack("Missing ET", ""); 
stack_lep_pt = R.THStack("Lepton pT", ""); 
stack_dilep1 = R.THStack("Invariant mass 2l", "");
stack_dilep2 = R.THStack("Invariant mass 2l", "");

In [86]:
for h in Backgrounds: 
    stack_mll.RecursiveRemove(H_mll[h]); ## Remove previously stacked histograms  
    
    stack_lep_pt.RecursiveRemove(H_lep_pt[h]);
    stack_dilep1.RecursiveRemove(H_dilep1[h]);
    stack_dilep2.RecursiveRemove(H_dilep2[h]);

    stack_mll.Add(H_mll[h]); 
    
    stack_lep_pt.Add(H_lep_pt[h]); 
    stack_dilep1.Add(H_dilep1[h]);
    stack_dilep2.Add(H_dilep2[h]);

Now we make a legend (see [TLegend](https://root.cern.ch/doc/master/classTLegend.html)), and add  the different backgrounds. Next we make a canvas (see [TCanvas](https://root.cern.ch/doc/master/classTCanvas.html)), which is allways necessary when we want to make a plot. Then you draw the stack and the legend, and display them by drawing the canvas. We can also specify axis labels and a bunch of other stuff. 

In [87]:
R.gStyle.SetLegendBorderSize(0); ## Remove (default) border around legend 
leg = R.TLegend(0.65, 0.60, 0.9, 0.85); 

In [88]:
leg.Clear();
for i in Backgrounds: 
    leg.AddEntry(H_mll[i], i, "f")  ## Add your histograms to the legend
leg.AddEntry(hist_mll_d, "Data", "lep") 

In [89]:
C = R.TCanvas("c", "c", 600, 600)

Warning in <TCanvas::Constructor>: Deleting canvas with same name: c


In [90]:
#R.gPad.SetLogy() ## Set logarithmic y-axis

In [91]:
hist_mll_d.SetLineColor(R.kBlack); 
hist_mll_d.SetMarkerStyle(R.kFullCircle); 
hist_mll_d.SetMarkerColor(R.kBlack); 

In [92]:
stack_mll.Draw("hist"); 
stack_mll.SetMaximum(30); 
stack_mll.SetMinimum(1); 
stack_mll.GetYaxis().SetTitle("# events");
stack_mll.GetYaxis().SetTitleOffset(1.3); 
stack_mll.GetXaxis().SetTitle("m_{llll} (GeV)");
stack_mll.GetXaxis().SetTitleOffset(1.3);
hist_mll_d.Draw("same E"); 
leg.Draw();
C.SaveAs("mlll.pdf")
C.Draw();

Info in <TCanvas::Print>: pdf file mlll.pdf has been created


In [93]:
hist_lep_pt_d.SetLineColor(R.kBlack); 
hist_lep_pt_d.SetMarkerStyle(R.kFullCircle); 
hist_lep_pt_d.SetMarkerColor(R.kBlack); 

In [94]:
stack_lep_pt.Draw("hist"); 
stack_lep_pt.SetMaximum(700); 
stack_lep_pt.GetYaxis().SetTitle("# events");
stack_lep_pt.GetYaxis().SetTitleOffset(1.3); 
stack_lep_pt.GetXaxis().SetTitle("p_{T} (GeV)");
stack_lep_pt.GetXaxis().SetTitleOffset(1.3);
hist_lep_pt_d.Draw("same e"); 
leg.Draw();
C.SaveAs("pT.pdf")
C.Draw(); 

Info in <TCanvas::Print>: pdf file pT.pdf has been created


In [95]:
hist_dilep1_d.SetLineColor(R.kBlack); 
hist_dilep1_d.SetMarkerStyle(R.kFullCircle); 
hist_dilep1_d.SetMarkerColor(R.kBlack); 

In [96]:
stack_dilep1.Draw("hist"); 
stack_dilep1.SetMaximum(100); 
stack_dilep1.SetMinimum(1); 
stack_dilep1.GetYaxis().SetTitle("# events");
stack_dilep1.GetYaxis().SetTitleOffset(1.3); 
stack_dilep1.GetXaxis().SetTitle("m_{ll1} (GeV)");
stack_dilep1.GetXaxis().SetTitleOffset(1.3);
hist_dilep1_d.Draw("same E"); 
leg.Draw();
C.SaveAs("mll1.pdf")
C.Draw();

Info in <TCanvas::Print>: pdf file mll1.pdf has been created


In [97]:
hist_dilep2_d.SetLineColor(R.kBlack); 
hist_dilep2_d.SetMarkerStyle(R.kFullCircle); 
hist_dilep2_d.SetMarkerColor(R.kBlack); 

In [98]:
stack_dilep2.Draw("hist"); 
stack_dilep2.SetMaximum(80); 
stack_dilep2.SetMinimum(1); 
stack_dilep2.GetYaxis().SetTitle("# events");
stack_dilep2.GetYaxis().SetTitleOffset(1.3); 
stack_dilep2.GetXaxis().SetTitle("m_{ll2} (GeV)");
stack_dilep2.GetXaxis().SetTitleOffset(1.3);
hist_dilep2_d.Draw("same E"); 
leg.Draw();
C.SaveAs("mll2.pdf")
C.Draw();

Info in <TCanvas::Print>: pdf file mll2.pdf has been created
